In [ ]:
%matplotlib inline

from wordcloud import *
import math
#from nltk import tokenize
#from nltk.corpus import stopwords
from collections import Counter
import random
import numpy as np
import string
import re
import os
import operator
from scipy.misc import imread
#import textblob
import pandas as pd
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import plotly as py
import psycopg2
import seaborn as sbn
import requests
import textatistic
from bs4 import BeautifulSoup
matplotlib.style.use('ggplot')
py.offline.init_notebook_mode()

In [ ]:
orders = pd.read_csv('/home/faculty/millbr02/pub/ecomm/orders.csv')

In [ ]:
products = pd.read_csv('/home/faculty/millbr02/pub/ecomm/products.csv')

In [ ]:
aisles = pd.read_csv('/home/faculty/millbr02/pub/ecomm/aisles.csv')

In [ ]:
op = pd.read_csv('/home/faculty/millbr02/pub/ecomm/order_products__prior.csv')

In [ ]:
departments = pd.read_csv('/home/faculty/millbr02/pub/ecomm/departments.csv')

In [ ]:
ot = pd.read_csv('/home/faculty/millbr02/pub/ecomm/order_products__train.csv')

In [ ]:
orders.head()

In [ ]:
products.head()

In [ ]:
departments.head(10)

In [ ]:
op.head()

In [ ]:
ot.head()

What is the most popular first item to add to the cart? Does that change by day of the week?

In [ ]:
ot[ot.add_to_cart_order == 1].groupby(ot.product_id)['product_id'].count().sort_values(ascending=False).head(1)

In [ ]:
products[products.product_id == 24852]

In [ ]:
merged = pd.merge(ot, products)

In [ ]:
merged[merged.add_to_cart_order == 1].groupby('product_id').count()

In [ ]:
beast = ot.groupby('order_id')['add_to_cart_order'].max()

In [ ]:
beast = pd.DataFrame(beast)
beast = beast.reset_index()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10,10))

axes.violinplot(beast['add_to_cart_order'], showmeans=True, showmedians=False)

In [ ]:
little_guy = ot.head(100)

In [ ]:
pt = pd.pivot_table(little_guy[little_guy.order_id<50], index='order_id', columns='product_id', values='add_to_cart_order')

In [ ]:
pt

In [ ]:
pt.iloc[0][pt.iloc[0].notnull()]

In [ ]:
t = ot.sort_values('order_id')
top = t.head(5000)

In [ ]:
product_dict = {}

In [ ]:
top_group = top.groupby('order_id')

In [ ]:
tlist = list(top_group.groups.keys())

In [ ]:
for item in tlist:
    lst = pd.DataFrame(top_group.get_group(item)['product_id'])
    product_list = []
    for i in range(len(lst.index)):
        product_list.append(lst.iloc[i].product_id)
    for product in product_list:
        if product not in product_dict:
            product_dict[product] = {}
            for other_product in product_list:
                if other_product != product:
                    product_dict[product][other_product] = 1
        else:
            for other_product in product_list:
                if other_product != product:
                    if other_product not in product_dict[product]:
                        product_dict[product][other_product] = 1
                    else:
                        product_dict[product][other_product] += 1

In [ ]:
print(product_dict)

In [ ]:
products.head()

In [ ]:
def suggested_products(product_id):
    suggest = []
    id_dict = product_dict[product_id]
    sorted_list = sorted(id_dict.items(), key=operator.itemgetter(1))
    while sorted_list != [] and len(suggest) <= 5:
        purchased = sorted_list.pop()
        if purchased[1] > 1:
            should_buy = products[products.product_id == purchased[0]]['product_name']
            for item in should_buy:
                suggest.append(item)
    return suggest

In [ ]:
print(products[products.product_id == 24852]['product_name'])

In [ ]:
suggested_products(24852)

In [ ]:
top['suggested_products'] = top.product_id.map(suggested_products)